# [Ultrafeedback](https://arxiv.org/abs/2310.01377)

O Ultrafeedback é um dataset gerado a partir de uma dataset sintético 
com 1,4 milhões de instruções, geração de respostas por LLMs maiores e anotação do GPT-4.

Um experimento interessante de fazer com o Ultrafeedback é usar o dataset para treinar modelos 
consideravelmente menores, como o Phi-1.5, usando o GRPO/PPO para melhorar a qualidade das respostas.


In [1]:
from datasets import load_dataset

ds = load_dataset("openbmb/UltraFeedback")

c:\dev\Track_RLAIF\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\dev\Track_RLAIF\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\moura\.cache\huggingface\hub\datasets--openbmb--UltraFeedback. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this

# Estrutura dos dados

- source: dataset de referência
    - Evol Instruct: 100% Helpful
    - FalseQA: 100% TruthfulQA
    - Flan: 60% Helpful, 20% Truthful, 20% Verbalized Calibration
    - ShareGPT: 60% Helpful, 20% Truthful, 18% Honesty, 2% Verbalized Calibration
    - TruthfulQA:	100% Truthful
    - UltraChat: 60% Helpful, 20% Truthful, 18% Honesty, 2% Verbalized Calibration

- instruction: Instrução para o LLM
- models: Modelo que gerou a resposta
- completions: Trás vários elementos, em "response" temos a resposta do modelo, temos também as anotações em "annotations" (helpfulness, honesty, instruction_following, truthfulness) e crítica em "critique". Cada anotação também apresenta uma nota de rating 1 a 5. 
- correct_answers - Nulo
- incorrect_answers - Nulo

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['source', 'instruction', 'models', 'completions', 'correct_answers', 'incorrect_answers'],
        num_rows: 63967
    })
})

In [7]:
example = ds["train"][0]
print("Instruction:\n", example["instruction"])
print("Models:\n", example["models"])
print("Anotações do :\n", example["completions"])

Instruction:
 Can you write a C++ program that prompts the user to enter the name of a country and checks if it borders the Mediterranean Sea? Here's some starter code to help you out:
#include <iostream>
#include <string>
using namespace std;
int main() {
    string country;
    // prompt user for input
    cout << "Enter the name of a country: ";
    cin >> country;
    // check if country borders the Mediterranean Sea
    // [C++ code]
    return 0;
}
Models:
 ['alpaca-7b', 'pythia-12b', 'starchat', 'vicuna-33b']
Completions:
 [{'annotations': {'helpfulness': {'Rating': '2', 'Rationale': 'The response is clear and not lengthy, but it lacks useful and comprehensive information.', 'Rationale For Rating': 'The code is partially incorrect as it checks if the country name ends with "Mediterranean" instead of checking if it borders the Mediterranean Sea, which may cause confusion.', 'Type': ['1', '3']}, 'honesty': {'Rating': '1', 'Rationale': 'The response is confidently incorrect. The pr

# Preparação dos dados

Vamos preparar uma amostra curta do Ultrafeedbacks, extraindo de completions todos as features

In [11]:
def process_one_example(example):
    """
    Reorganiza as informações de 'completions' em uma estrutura limpa.
    """
    processed = {
        "instruction": example.get("instruction", None),
        "completions": []
    }
    
    for comp in example.get("completions", []):
        item = {
            "model": comp.get("model"),
            "response": comp.get("response"),
            "overall_score": comp.get("overall_score"),
            "fine_grained_score": comp.get("fine-grained_score"),
            "critique": comp.get("critique"),
            "annotations": {}
        }
        
        # reorganiza as anotações
        for ann_name, ann_data in comp.get("annotations", {}).items():
            item["annotations"][ann_name] = {
                "rating": ann_data.get("Rating"),
                "rationale": ann_data.get("Rationale"),
                "rationale_for_rating": ann_data.get("Rationale For Rating", None),
                "types": ann_data.get("Type", [])
            }
        
        processed["completions"].append(item)
    
    return processed

import pandas as pd

def example_to_df(clean_example):
    """
    Converte um exemplo processado em um DataFrame.
    Cada completion vira uma linha.
    """
    rows = []
    instr = clean_example.get("instruction", None)
    
    for comp in clean_example.get("completions", []):
        base = {
            "instruction": instr,
            "model": comp.get("model"),
            "response": comp.get("response"),
            "overall_score": comp.get("overall_score"),
            "fine_grained_score": comp.get("fine_grained_score"),
            "critique": comp.get("critique"),
        }
        
        # adicionar anotações como colunas flatten
        for ann_name, ann_data in comp.get("annotations", {}).items():
            base[f"{ann_name}_rating"] = ann_data.get("rating")
            base[f"{ann_name}_rationale"] = ann_data.get("rationale")
        
        rows.append(base)
    
    return pd.DataFrame(rows)


example = ds["train"][0]
clean = process_one_example(example)
df = example_to_df(clean)
df.to_csv("ultrafeedbacks_sample.csv", index=False)